In [1]:
import os
import pandas as pd
import yaml
import copy
import random
from pathlib import Path
import matplotlib.pyplot as plt
from copy import deepcopy

import aot

In [2]:
base_dir = Path(aot.__path__[0])
core_expt_yaml_path = base_dir / "experiment/core_exp_settings.yml"
stimuli_temp_path = base_dir / "experiment/stimuli_settings_temp.yml"
core_settings = yaml.load(open(core_expt_yaml_path), Loader=yaml.FullLoader)
stimuli_settings_temp = yaml.load(open(stimuli_temp_path), Loader=yaml.FullLoader)

settings_root_path = base_dir / core_settings["paths"]["settings_path"]
video_db_path = base_dir / "data/videos/database_originals.tsv"
video_db = pd.read_csv(video_db_path, sep="\t")

main_settings_dir = base_dir / core_settings["paths"]["settings_path"] / "main"
memory_settings_dir = base_dir / core_settings["paths"]["settings_path"] / "memory_main"
subject_number = core_settings["various"]["subject_number"]
run_number = core_settings["various"]["run_number"]
session_number = core_settings["various"]["session_number"]
unique_video_number_per_session = core_settings["various"][
    "unique_video_number_per_session"
]
img_number_per_session = unique_video_number_per_session
total_video_number = core_settings["various"]["total_video_number"]

original_video_pure_names = []
for i in range(1, 1 + total_video_number):
    # 0 padding for video name
    video_name = str(i).zfill(4)
    original_video_pure_names.append(video_name)
print("videos number: ", len(original_video_pure_names))

videos number:  2214


In [3]:
def get_session_used_video_purenames(subject, session):
    """
    get all the images that corresponding to the videos used in a session
    """
    main_runs_settings = []
    main_runs_videos = []
    main_runs_videos_pure_name = []  # raw video name without .mp4
    for ind in range(run_number):
        run = ind + 1
        runfile = f"experiment_settings_sub_{str(subject).zfill(2)}_ses_{str(session).zfill(2)}_run_{str(run).zfill(2)}.yml"
        runfile = main_settings_dir / runfile
        runsetting = yaml.load(open(runfile), Loader=yaml.FullLoader)
        main_runs_settings.append(runsetting)
    for runsetting in main_runs_settings:
        raw_video_list = runsetting["stimuli"]["movie_files"]
        video_list = [video for video in raw_video_list if video != "blank"]
        # print(video_list)
        main_runs_videos.append(video_list)
    for videos in main_runs_videos:
        # only pure name, without .mp4
        name_list = [video.split(".")[0] for video in videos]
        main_runs_videos_pure_name.append(name_list)
        # print(name_list)
    # flatten the videos list
    flat_video_pure_names = [
        item for sublist in main_runs_videos_pure_name for item in sublist
    ]
    # remove suffix
    flat_video_pure_names = [
        video.replace("_rv", "") for video in flat_video_pure_names
    ]
    flat_video_pure_names = [
        video.replace("_fw", "") for video in flat_video_pure_names
    ]
    # remove repeated videos
    flat_video_pure_names = list(set(flat_video_pure_names))
    # print(flat_video_names)

    # maybe add some extra steps to remove all the prefix or suffix for reverse and resample, to get names that correspond to the
    # origina video names

    print(f"sub {subject} ses {session} used videos pure names:",flat_video_pure_names)
    return flat_video_pure_names

In [4]:
def get_session_unused_video_purenames(subject,session):
    '''
    get all the images that corresponding to the videos unused in a session,
    first get all the used videos names the filter them out from all the video names to get unused video names
    '''
    used_video_pure_names = set(get_session_used_video_purenames(subject,session))
    print("used_video_pure_names",used_video_pure_names)
    all_video_names = set(original_video_pure_names)
    print("all_video_names",all_video_names)
    unused_video_names = list(all_video_names - used_video_pure_names)
    print(f"sub {subject} ses {session} unused videos pure names:",unused_video_names)
    return unused_video_names


In [5]:
def gen_memory_design(subject, session):
    print(f"subject: {subject}, session: {session}")
    # read main experiment settings
    used_images_purenames = get_session_used_video_purenames(subject, session)
    ununsed_images_purenames = get_session_unused_video_purenames(subject, session)

    # maybe different ways to add suffix to get the image names
    used_images_names = [name + ".png" for name in used_images_purenames]
    ununsed_images_names = [name + ".png" for name in ununsed_images_purenames]
    print("len used images:",len(used_images_names))
    print("len unused images:",len(ununsed_images_names))

    # save imgnames as settings for memory experiment yaml file
    settings = settings = copy.deepcopy(stimuli_settings_temp)

    half_img_num = int((img_number_per_session) / 2)
    # sample half_img_num images from imgnames
    imgnames_used = random.sample(used_images_names, half_img_num)
    imgnames_unused = random.sample(ununsed_images_names, half_img_num) 
    imgnames_final = imgnames_used + imgnames_unused
    random.shuffle(imgnames_final)

    settings["stimuli"][
        "picture_files"
    ] = imgnames_final  ##not right. should subsample from imgnames and add some img from unused videos
    print(settings)
    outfile = f"experiment_settings_sub_{str(subject).zfill(2)}_ses_{str(session).zfill(2)}_run_01.yml"
    outfile = memory_settings_dir / outfile
    with open(outfile, "w") as f:
        yaml.dump(settings, f)

In [6]:
def gen_memory_desing_for_all():
    for sub in range(1,subject_number+1):
        for ses in range(1,session_number+1):
            gen_memory_design(sub, ses)       

In [7]:
gen_memory_desing_for_all()

subject: 1, session: 1
sub 1 ses 1 used videos pure names: ['0297', '0837', '1360', '1061', '0342', '1746', '0421', '1847', '1750', '2179', '1450', '1692', '0808', '2123', '1334', '1542', '1209', '0200', '1060', '2068', '1583', '2190', '2006', '2003', '1066', '0664', '1700', '1982', '1570', '1744', '0773', '1520', '1881', '0121', '1857', '1141', '1537', '2086', '1661', '0581', '2037', '2005', '1599', '1751', '2058', '0857', '1890', '1445', '0877', '1125', '0050', '2078', '1779', '2160', '1821', '1253', '2002', '1481', '1250', '2141', '0842', '0040', '1350', '1355', '1372', '1868', '1234', '1713', '0215', '1328', '0402', '0725', '1104', '1559', '2048', '0602', '2124', '1557', '1170', '1668', '1459', '1742', '1279', '2202', '0353', '0717', '1601', '1710', '1438', '1122', '1827', '1647', '0904', '0756', '0459', '0953', '0158', '0714', '2077', '1035', '0940', '2157', '1788', '1818', '0390', '1503', '1152', '0063', '1307', '0666', '0406', '2143', '1497', '0295', '0086', '1449', '1962', '067